# Simulation run

This notebook takes the data, which has been imputed and had constructions added, and uses SimStock to run a simulation.

First, the data is read in a as a SimStockDataframe

In [ ]:
import simstock as sim
import matplotlib.pyplot as plt

# Read in the data as a SimsockDataframe
sdf = sim.read_csv(
    "clustered.csv",
    uid_column_name="SCU",
    use_base_idf=True,
    idd_file="/Applications/EnergyPlus-8-9-0/Energy+.idd" # Specify path to EnergyPlus here
    )

# Set the constructions to be the clusters
sdf["construction"] = sdf["Cluster"]

# Plot it
sim.plot(sdf)
plt.show()

We now use the csv files in the settings folder to specify simulation settings. This is making use of SimStock's spreadsheet editing interfact: see online docs.

In [ ]:
sdf.override_settings_with_csv(settings_csv_path="data/settings")
sdf.preprocessing()

Check what the output variables are.

In [ ]:
print(sdf.output_variable)

Change the last one to zone outdoor air temperature.

In [ ]:
tmp = sdf.output_variable[2]
tmp.Variable_Name = "Zone Outdoor Air Temperature"
tmp.Reporting_Frequency = "Hourly"

print(sdf.output_variable)

Now we add some more output variables:

In [ ]:
new_d = {"Key_Value": "*", "Variable_Name": "Zone Ideal Loads Zone Total Cooling Energy", "Reporting_Frequency": "Hourly"}
sdf.settings.newidfobject("Output:Variable", **new_d)
new_d = {"Key_Value": "*", "Variable_Name": "Zone Ideal Loads Zone Total Heating Energy", "Reporting_Frequency": "Hourly"}
sdf.settings.newidfobject("Output:Variable", **new_d)
new_d = {"Key_Value": "*", "Variable_Name": "Zone Mean Air Temperature", "Reporting_Frequency": "Hourly"}
sdf.settings.newidfobject("Output:Variable", **new_d)
sdf.settings.idfobjects["OUTPUT:METER"][0].Reporting_Frequency = "Hourly"

At this point, we wil save the simstock daataframe to csv for prosperity.

In [ ]:
sdf._df.to_csv("data/preprocessed_croydon.csv")

Now we set the simulation timestep and print some information about the simulation we're about to run.

In [ ]:
# Set timestep
sdf.timestep = 4

# Print some info
print(f"Number of SCUs: {sdf.length}")
print(f"Number of built islands: {sdf.built_islands}")
print(f"Weather file: {sdf.epw}")
print(f"Are all the geometries valid? {sdf.is_valid.all()}")
print(f"Timestep: {sdf.timestep}")
print(f"Run period: {sdf.run_period}")

We'll now go ahead and create the EnergyPlus input data files, and then save them.

In [ ]:
simulation = sim.IDFmanager(sdf)
simulation.create_model_idf()
simulation.save_idfs(out_dir="model_idfs_rerun_2")

And finally, run the simulation.

In [ ]:
simulation.run()